In [29]:
import pandas as pd
import numpy as np
import random
import os

df=pd.read_csv('/home/bigdata/Task_1.4_Data_Wrangling_Challenge_“DirtyData”_Exercise/datacreator/source_clean_dataset.csv')
df.head()


,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [30]:
# Columnas seleccionadas para el ejercicio (descartamos 'Unnamed: 0')
columns_to_keep = ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'yr_built', 'condition']

# 1. Tomar una muestra aleatoria de N_ROWS filas y seleccionar las columnas
df = df[columns_to_keep].sample(n=500, random_state=42).copy()
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'yr_built', 'condition'],
      dtype='object')

# 1.Missing data: Remove or leave blank some values randomly.


In [31]:
# Porcentaje de valores a convertir a NaN (Not a Number)
MISSING_PERCENT_PRICE = 0.05  # 5%
MISSING_PERCENT_SQFT = 0.10   # 10%

# 1. Aplicar NaN a la columna 'price' (5% de las filas)
# Seleccionamos aleatoriamente los índices a modificar
mask_missing_price = df.sample(frac=0.05, random_state=1).index
df.loc[mask_missing_price, 'price'] = np.nan

# 2. Aplicar NaN a la columna 'sqft_living' (10% de las filas)
mask_missing_sqft = df.sample(frac=0.10, random_state=2).index
df.loc[mask_missing_sqft, 'sqft_living'] = np.nan
# Opcional: Verificar cuántos NaN se agregaron
print(df[['price', 'sqft_living']].isnull().sum())

price          25
sqft_living    50
dtype: int64


In [32]:
import numpy as np
import random

# Data String vacio

# Seleccionamos aleatoriamente 1% de las filas
mask_empty_str = df.sample(frac=0.01, random_state=3).index
df.loc[mask_empty_str, 'bathrooms'] = ""

# Data 'N/A' en bedrooms
mask_na_str = df.sample(frac=0.005, random_state=4).index
df.loc[mask_na_str, 'bedrooms'] = "N/A"

/tmp/ipykernel_41340/3971685450.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask_empty_str, 'bathrooms'] = ""
/tmp/ipykernel_41340/3971685450.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[mask_na_str, 'bedrooms'] = "N/A"


# 2.Duplicated rows: Add exact or partial duplicates.



In [33]:
df_duplicated = df.sample(frac=0.2, random_state=123)
df = pd.concat([df, df_duplicated], ignore_index=True)

# 3.Outliers: Insert extreme values in numeric columns.



In [34]:
sample_outliers_magnitude = df.sample(n=4, random_state=12).index

# Precios muy altos
df.loc[sample_outliers_magnitude, 'price'] = 1000000000000

# Superficie enorme
df.loc[sample_outliers_magnitude, 'sqft_living'] = 1000000


#Fechas imposibles
sample_outliers_year = df.sample(n=5, random_state=13).index
df.loc[sample_outliers_year[:3], 'yr_built'] = 2050
df.loc[sample_outliers_year[3:], 'yr_built'] = 1700

# Habitaciones aleatorias
sample_outliers_count = df.sample(n=3, random_state=14).index
df.loc[sample_outliers_count, 'bedrooms'] = 50


# 4.Format inconsistencies: Change date formats, numeric formats, or units.



In [35]:
sample_format_price = df.sample(frac=0.01, random_state=15).index
# Esto convierte la columna 'price' a tipo 'object' (string)
df.loc[sample_format_price, 'price'] = ('$' + df.loc[sample_format_price, 'price'].round(2).astype(str))

/tmp/ipykernel_41340/3022116623.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['$369900.0' '$308000.0' '$335000.0' '$403950.0' '$296000.0' '$nan']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[sample_format_price, 'price'] = ('$' + df.loc[sample_format_price, 'price'].round(2).astype(str))


# 5.Typographical errors: Introduce spelling mistakes in categorical columns.



In [36]:

# 2. Reemplazamos TODAS las instancias de la categoría '4' por el error tipográfico 'Fouur'
df['condition'] = df['condition'].astype(str)
df.loc[df['condition'] == '4', 'condition'] = 'Fouur'
print(df['condition'].unique())

['5' '3' 'Fouur' '1' '2']


# 6.Extra categories: Add unusual values in categorical columns to simulate errors.



In [37]:
sample_error = df.sample(frac=0.06, random_state=555).index
error_choices = ['ZOMBIE', 'ERROR_MAP', 'NA_JUNK', '999'] # Valores basura

df.loc[sample_error, 'condition'] = np.random.choice(error_choices, size=len(sample_error))
print(df['condition'].unique())

['5' '3' 'Fouur' 'ERROR_MAP' 'ZOMBIE' '999' 'NA_JUNK' '1' '2']


# 7.Incorrect data types: Store numbers as strings or vice versa.



In [38]:
sample_year_string = df.sample(frac=0.05, random_state=777).index
df.loc[sample_year_string, 'yr_built'] = df.loc[sample_year_string, 'yr_built'].astype(int).astype(str) + ' Built'

/tmp/ipykernel_41340/3454239482.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2050 Built' '1965 Built' '1987 Built' '1992 Built' '1951 Built'
 '1953 Built' '1968 Built' '1978 Built' '1993 Built' '1961 Built'
 '1964 Built' '1952 Built' '1914 Built' '1963 Built' '1986 Built'
 '1992 Built' '1989 Built' '1954 Built' '1960 Built' '1978 Built'
 '1994 Built' '1998 Built' '1918 Built' '2005 Built' '1922 Built'
 '1985 Built' '1934 Built' '1963 Built' '1911 Built' '1982 Built']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[sample_year_string, 'yr_built'] = df.loc[sample_year_string, 'yr_built'].astype(int).astype(str) + ' Built'


# 8.No default codification of file (utf-8?)


In [39]:

def encoding_chars(condition_str):
    # Solo aplicamos si el valor no es NaN
    if pd.notna(condition_str) and isinstance(condition_str, str):
        # Reemplazamos 'a' por 'á' (solo la primera instancia)
        condition_str = condition_str.replace('a', 'á', 1) 
        # Reemplazamos 'e' por 'é' (solo la primera instancia)
        condition_str = condition_str.replace('e', 'é', 1)
    return condition_str

sample_special_char_date = df.sample(frac=0.01, random_state=999).index
df.loc[sample_special_char_date, 'date'] = 'ERROR-Año-Ñ' + df.loc[sample_special_char_date, 'date'].astype(str).str[-4:]
print(df.loc[sample_special_char_date, 'date'].head())

428    ERROR-Año-Ñ0000
566    ERROR-Año-Ñ0000
511    ERROR-Año-Ñ0000
103    ERROR-Año-Ñ0000
86     ERROR-Año-Ñ0000
Name: date, dtype: object


# 9.Incorrect headers


In [40]:
# Diccionario para mapear los nombres de columna existentes a los nuevos nombres "sucios"
new_columns_corrected = {
    'id': 'ID_REGISTRO_X',       # 'id' -> ID_REGISTRO_X
    'date': 'date_',                # 'date' -> date_
    'price': 'price',       
    'bedrooms': 'bedrooms',        
    'bathrooms': 'BATH',         # 'bathrooms' -> BATH
    'sqft_living': 'sqft_living', 
    'yr_built': 'yr_built', 
    'condition': 'condition'  
}

# Aplicamos el renombrado
df.rename(columns=new_columns_corrected, inplace=True)

print("Nuevos encabezados con inconsistencias:")
print(list(df.columns))

Nuevos encabezados con inconsistencias:
['ID_REGISTRO_X', 'date_', 'price', 'bedrooms', 'BATH', 'sqft_living', 'yr_built', 'condition']


# 10.Extra punctuation symbols (1000€)

In [41]:
sample_format_error = df['price'].sample(frac=0.15, random_state=222).index

def format_as_euro_suffix(value):
    if pd.notna(value) and isinstance(value, (int, float)):
        return f"{value:.0f} EUR" 
    return value

# Aplicar a 'price'
df.loc[sample_format_error, 'price'] = df.loc[sample_format_error, 'price'].apply(format_as_euro_suffix)
print(df['price'].dtype)

object


In [42]:
df.to_csv('dirty_dataset.csv', index=False, encoding='latin-1')

📝 Informe sobre la Inserción de Errores en el Dataset de Viviendas (Housing Data)
Este informe detalla las técnicas utilizadas para ensuciar intencionadamente el dataset de ventas de casas, con el fin de crear un escenario realista de "Data Cleaning".
El dataset original se redujo inicialmente a una muestra aleatoria de 500 filas. Posteriormente, se añadió un 20% de filas duplicadas.
Detalles de la Implementación por Punto
1. Datos Faltantes (Missing Data)Se introdujeron valores nulos (NaN) y vacíos en varias columnas clave para simular registros incompletos:Price: Se reemplazó el 5% de los valores con NaN (nulos numéricos).Sqft_living: Se reemplazó el 10% de los valores con NaN.Bathrooms: Se reemplazó el 1% con un string vacío ("").Bedrooms: Se reemplazó el 0.5% con el string 'N/A'.
2. Filas Duplicadas (Duplicated Rows)Se tomó una muestra aleatoria del 20% de las filas existentes y se concatenó al dataset, generando duplicados exactos.
3. Valores Atípicos/Extremos (Outliers)Se insertaron valores ilógicos o extremos para romper las estadísticas:Price: Se asignó un valor de 1 billón (1,000,000,000,000) a 4 filas.Sqft_living: Se asignó un valor de 1,000,000 (un millón de pies cuadrados) a las mismas filas.Yr_built (Año de construcción): Se cambiaron 3 valores a 2050 (futuro) y 2 valores a 1700 (demasiado antiguo).Bedrooms: Se asignaron 50 habitaciones a 3 filas aleatorias.
4. Inconsistencias de FormatoPrice (Moneda): En el 1% de los datos, se convirtió el precio numérico a texto añadiendo el símbolo $ al principio (ej: "$369900.0"), convirtiendo la columna en tipo object.
5. Errores Tipográficos (Typographical Errors)Condition: Se reemplazaron todas las instancias de la categoría '4' por la palabra mal escrita 'Fouur'.
6. Categorías Extranormales (Extra Categories)Condition: Se inyectó "basura" en el 6% de los registros de esta columna. Los valores insertados aleatoriamente fueron: 'ZOMBIE', 'ERROR_MAP', 'NA_JUNK', y '999'.
7. Tipo de Dato Incorrecto (Incorrect Data Types)Yr_built: Se convirtió el 5% de los años a texto añadiendo la palabra "Built" (ej: '1992 Built'), rompiendo el formato numérico de la columna.
8. Problemas de Codificación (Encoding Issues)Date: En el 1% de las filas, se modificó la fecha añadiendo el prefijo 'ERROR-Año-Ñ', introduciendo caracteres especiales (Ñ) que fuerzan problemas si no se usa la codificación correcta.
9. Encabezados Incorrectos (Incorrect Headers)Se renombraron varias columnas para simular inconsistencias en la nomenclatura:id $\rightarrow$ ID_REGISTRO_Xdate $\rightarrow$ date_bathrooms $\rightarrow$ BATH
10. Símbolos de Puntuación/Moneda ExtraPrice (Sufijo): Además del error del punto 4, en otro 15% de los datos se añadió el sufijo ' EUR' al precio (ej: '450000 EUR'), mezclando números y texto en la columna principal de valor.